#Весь код 

In [ ]:
# !pip install beautifulsoup4
from bs4 import BeautifulSoup
import urllib.request
import tqdm
import pandas as pd
import re


def gethrefs():
  a_hrefs = []
  pages_range = range(2388) #2388
  for page_num in tqdm.tqdm(pages_range):
    url = "https://www.sports.ru/football/sportsman/?page={}".format(page_num)
    try: 
      page_files = urllib.request.urlopen(url)
    except:
      print('page_not_found')
      return None
    file_data = ""
    for line in page_files:
      decoded_line = line.decode("utf-8")
      file_data += decoded_line
    soup = BeautifulSoup(file_data, 'html.parser') #make soup that is parse-able by bs

    for div in soup.find_all('div', class_='overBox' ):
      for a_tag in div.find_all('a', class_='name'):
        a_str = a_tag.get('href')
        a_str += 'career'
        a_hrefs.append(a_str)

  return a_hrefs



def parse_page(url):
  player_name = 'None'
  followers = '0'

  try: 
    page_files = urllib.request.urlopen(url)
  except:
    print('page_not_found')
    return None
  file_data = ""
  for line in page_files:
    decoded_line = line.decode("utf-8")
    file_data += decoded_line
  soup = BeautifulSoup(file_data, 'html.parser')
  name_div = soup.find_all('div', class_='descr')
  # followers = soup.find_all('span', class_='count-less') 
  followers_span = soup.find("span", class_="count")
  
  # for f in followers_span:
  try:
    followers = followers_span.text
  except:
    print('err in followers')
    followers = '0'  
# name_str = soup.find('div').getText()
  for n in name_div:
    player_name = n.text
  try:
    df_football_player2 = pd.read_html(url)
  except:
    return None
  last_table = df_football_player2[-1].iloc[[-1]]

  info_df2 = df_football_player2[0].T
  info_df2 = info_df2.iloc[[1]]

  last_table.insert(0, "Name", player_name, True)
  last_table = last_table.reset_index()
  last_table.insert(1, "Followers", followers, True)
  last_table = last_table.reset_index()
  last_table.drop([0], axis=1, inplace=True)

  info_df2 = info_df2.reset_index()
  res_df = pd.concat([last_table, info_df2], axis=1, ignore_index=True)
  res_df.drop([0], axis=1, inplace=True)
  res_df.columns = range(res_df.shape[1])
  return res_df


def return_age(df, col_n=12):
  st = df.iloc[0,col_n]
  data_birth = st.split('|')
  age = data_birth[-1]
  match = re.search(r'\d\d', age)
  try:
    age = match.group()
  except:
    print('age none')
    return 'None'
  return age


def return_weight_height(df):
  el = df.iloc[0,-1]
  el_arr = el.split('|')
  if len(el_arr) == 1:
    return el_arr, 'None'
  else:
    
    weight = el_arr[1]
    height = el_arr[0]
  # match = re.search(r'\d\d', age)
  # age = match.group()
    return height, weight


def return_club_position(df):
  n_columns = df.shape[1]
  stroka = df.iloc[0,n_columns-2].split('|')
  if len(stroka) == 1:
    club = 'None'
    return club, stroka
  else:
    club, position = stroka
    return club, position

#table = parse_page('https://www.sports.ru/tags/161085686/career')
#a_hrefs = gethrefs()

#with open('all_links.txt', 'w') as wr:
#  for el in a_hrefs:
#    wr.write(el + '\n')

a_hrefs = []
with open('all_links.txt', 'r') as rea:
  for el in rea.readlines():
    a_hrefs.append(el)

table_field_players = []
table_goalkeepers = []

for i in tqdm.tqdm(range(len(a_hrefs))):
  try:
    table = parse_page(a_hrefs[i])
  except:
    print('err in parsing')
    continue
  if table is None:
    continue
  #print(len(table.columns))
  if len(table.columns) == 21:
    age = return_age(table, col_n=17)
    height, weight = return_weight_height(table)
    club, position = return_club_position(table)
    # print(table)
    # table.iloc[0,17] = age
    table.insert(1, "Weight", weight, True)
    table.insert(2, "Height", height, True)
    table.insert(3, "Age", age, True)
    table.insert(4, "Club", club, True)
    table.insert(5, "Position", position, True)

    table_goalkeepers.append(table)
  elif len(table.columns) == 16:
    # print(table)
    age = return_age(table, col_n=12)
    height, weight = return_weight_height(table)
    club, position = return_club_position(table)
    # table.iloc[0,12] = age
    # table.iloc[0,-1] = height

    table.insert(1, "Weight", weight, True)
    table.insert(2, "Height", height, True)
    table.insert(3, "Age", age, True)
    table.insert(4, "Club", club, True)
    table.insert(5, "Position", position, True)

    # table = table.reset_index()
    table_field_players.append(table)

res_field_players = pd.concat(table_field_players, axis=0, ignore_index=True)
res_goalkeepers = pd.concat(table_goalkeepers, axis=0, ignore_index=True)

res_field_players.to_csv('res_field_players.csv', index=False, encoding='utf-8-sig')
res_goalkeepers.to_csv('res_goalkeepers.csv', index=False, encoding='utf-8-sig')

['https://www.sports.ru/thomas-muller/career', 'https://www.sports.ru/tags/161085686/career', 'https://www.sports.ru/xavi/career']


#Попытки

In [ ]:
u = 'https://www.sports.ru/tags/161085686/career'
table = parse_page(u)
if table:
  print(34)

page_not_found


In [ ]:
res_field_players.to_csv('res_field_players.csv')

In [ ]:
res_goalkeepers

,0,1,Weight,Height,Age,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,11,118269,82,186,35,Igor Akinfeev,549,0,0,1,31,1,514,0,217,299,107,142,90,1,8 апреля 1986 | 35 лет,Россия,ЦСКА|Вратарь,186|82
1,15,5551,92,192,43,Gianluigi Buffon,558,0,0,0,11,4,433,0,264,350,123,85,89,1,28 января 1978 | 43 года,Италия,Ювентус|Вратарь,192|92
2,18,4846,85,193,30,David De Gea Quintana,578,0,0,0,12,0,595,0,208,311,126,141,89,1,7 ноября 1990 | 30 лет,Испания,Манчестер Юнайтед|Вратарь,193|85
3,15,2961,85,187,28,Marc-Andre ter Stegen,424,0,0,2,10,0,408,0,166,268,82,74,90,1,30 апреля 1992 | 28 лет,Германия,Барселона|Вратарь,187|85
4,14,2927,92,193,35,Manuel Neuer,706,0,0,2,20,0,588,0,319,489,96,120,90,1,27 марта 1986 | 35 лет,Германия,Бавария|Вратарь,193|92
5,17,2864,79,185,39,Iker Casillas Fernández,719,0,0,1,33,2,634,0,277,487,104,128,90,1,20 мая 1981 | 39 лет,Испания,Вратарь,185|79


In [ ]:
url_addr = "https://www.sports.ru/buffon/career/"
url_addr2 = "https://www.sports.ru/tags/161048356/career"
res_df1 = parse_page(url_addr)
res_df2 = parse_page(url_addr2)

res_df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,15,-62,Gerson Rodrigues Correia Leal,197,50,7,4,54,38,3,65,1,20 июня 1995 | 25 лет,Люксембург,Динамо Киев|Полузащитник


#Постобработка футболисты + вратари


In [ ]:
field_players = pd.read_csv('res_field_players.csv', encoding='utf-8-sig')
nationality = field_players['13']
field_players.insert(6, "Nationality", nationality, True)
res_field_df = field_players.drop(columns=['11', '12', '13', '14', '15'])
res_field_df.head(1)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,Weight,Height,Age,Club,Position,Nationality,1,2,3,4,5,6,7,8,9,10
0,10.0,72,170,33,Барселона,Нападающий,Аргентина,352455,Lionel Andrés Messi,838,694,89.0,246.0,937.0,86.0,2.0,80.0


In [ ]:
columns_rename = {'0': 'Tournaments',
                  '1': 'Subscribers',
                  '2': 'Name',
                  '3': 'Matches',
                  '4': 'Goals',
                  '5': 'Penalties',
                  '6': 'Passes',
                  '7': 'Goal+Passes',
                  '8': 'Yellow Cards',
                  '9': 'Red Cards',
                  '10': 'Avg. min played',
                  }
field_df = res_field_df.rename(columns=columns_rename)
field_df.head(5)

,Tournaments,Weight,Height,Age,Club,Position,Nationality,Subscribers,Name,Matches,Goals,Penalties,Passes,Goal+Passes,Yellow Cards,Red Cards,Avg. min played
0,10.0,72,170,33,Барселона,Нападающий,Аргентина,352455,Lionel Andrés Messi,838,694,89.0,246.0,937.0,86.0,2.0,80.0
1,19.0,84,187,36,Ювентус,Нападающий,Португалия,347040,Cristiano Ronaldo dos Santos Aveiro,900,721,131.0,170.0,891.0,118.0,11.0,83.0
2,19.0,68,175,29,ПСЖ,Нападающий,Бразилия,210888,Neymar da Silva Santos Junior,480,295,48.0,141.0,436.0,127.0,9.0,82.0
3,14.0,73,178,22,ПСЖ,Нападающий,Франция,175892,Kylian Mbappe Lottin,262,168,11.0,75.0,243.0,25.0,3.0,71.0
4,24.0,95,195,39,Милан,Нападающий,Швеция,143586,Zlatan Ibrahimovic,673,450,82.0,125.0,575.0,112.0,14.0,80.0


In [ ]:
field_df.to_csv('field_players.csv', encoding='utf-8-sig')

In [ ]:
goalkeepers = pd.read_csv('res_goalkeepers.csv', encoding='utf-8-sig')
nationality_gk = goalkeepers['18']
goalkeepers.insert(6, "Nationality", nationality_gk, True)
res_goalkeepers = goalkeepers.drop(columns=['10','16', '17', '18', '19', '20'])
res_goalkeepers.head(1)

,0,Weight,Height,Age,Club,Position,Nationality,1,2,3,4,5,6,7,8,9,11,12,13,14,15
0,11,82,186,35,ЦСКА,Вратарь,Россия,118336,Igor Akinfeev,549,0,0,1,31,1,514,217,299,107,142,90


In [ ]:
columns_gk_rename = {'0': 'Tournaments',
                  '1': 'Subscribers',
                  '2': 'Name',
                  '3': 'Matches',
                  '4': 'Goals',
                  '5': 'Penalties',
                  '6': 'Passes',
                  '7': 'Yellow Cards',
                  '8': 'Red Cards',
                  '9': 'Conceded Goals',
                  '11': 'Dry matches',
                  '12': 'Wins',
                  '13': 'Draws',
                  '14': 'Losses',
                  '15': 'Avg. min played',
                  }
gk_df = res_goalkeepers.rename(columns=columns_gk_rename)
gk_df.head(5)

,Tournaments,Weight,Height,Age,Club,Position,Nationality,Subscribers,Name,Matches,Goals,Penalties,Passes,Yellow Cards,Red Cards,Conceded Goals,Dry matches,Wins,Draws,Losses,Avg. min played
0,11,82,186,35,ЦСКА,Вратарь,Россия,118336,Igor Akinfeev,549,0,0,1,31,1,514,217,299,107,142,90
1,15,92,192,43,Ювентус,Вратарь,Италия,5554,Gianluigi Buffon,558,0,0,0,11,4,433,264,350,123,85,89
2,18,85,193,30,Манчестер Юнайтед,Вратарь,Испания,4849,David De Gea Quintana,579,0,0,0,12,0,595,209,312,126,141,89
3,15,85,187,28,Барселона,Вратарь,Германия,2964,Marc-Andre ter Stegen,424,0,0,2,10,0,408,166,268,82,74,90
4,14,92,193,35,Бавария,Вратарь,Германия,2928,Manuel Neuer,706,0,0,2,20,0,588,319,489,96,120,90


In [ ]:
gk_df.to_csv('goalkeepers.csv', encoding='utf-8-sig')

#Парсинг клубов

In [ ]:
# !pip install beautifulsoup4
from bs4 import BeautifulSoup
import urllib.request
import tqdm
import pandas as pd
import re


def gethrefs(url_name, num=2):
  a_hrefs = []
  pages_range = range(2) #2388
  for page_num in tqdm.tqdm(pages_range):
    url_ = url_name +"{}"
    url = url_.format(page_num)
    
    try: 
      page_files = urllib.request.urlopen(url)
    except:
      print('page_not_found')
      return None
    file_data = ""
    for line in page_files:
      decoded_line = line.decode("utf-8")
      file_data += decoded_line
    soup = BeautifulSoup(file_data, 'html.parser') #make soup that is parse-able by bs

    for div in soup.find_all('div', class_='overBox' ):
      for a_tag in div.find_all('a', class_='name'):
        a_str = a_tag.get('href')
        # a_str += 'stat/'
        a_hrefs.append(a_str)

  return a_hrefs

def gethrefs_tournament(url_name):
  a_hrefs = []
  # print(url_name)
  try: 
    page_files = urllib.request.urlopen(url_name)
  except:
    print('page_not_found')
    return None
  file_data = ""
  for line in page_files:
    decoded_line = line.decode("utf-8")
    file_data += decoded_line
  soup = BeautifulSoup(file_data, 'html.parser') #make soup that is parse-able by bs
  for div in soup.find_all('ul', class_='options' ):
    for a_tag in div.find_all('a', class_='option'):
      a_str = a_tag.get('href')
      # a_str += 'stat/'
      a_hrefs.append(a_str)

  return a_hrefs

In [ ]:
import pandas as pd

def parse_club_page(url):
  try: 
    page_files = urllib.request.urlopen(url)
  except:
    print('page_not_found')
    return None, _, _

  keyword = "stat/"
  before_keyword, keyword, past_keyword = url.partition(keyword)
  season_year = past_keyword.split('/')[0]
  # print(season_year)

  file_data = ""
  for line in page_files:
    decoded_line = line.decode("utf-8")
    file_data += decoded_line
  soup = BeautifulSoup(file_data, 'html.parser') #make soup that is parse-able by bs

  for div in soup.find_all('h1', class_='titleH1' ):
    try:
      club_name = div.text.split(' - ')[0]
    except:
      return None, _, _
    try:
      league = str(div.text.split('статистика ')[1][:-5])
    except:
      return None, _, _
    # print(league)

  df_all = pd.read_html(url)
  city, country = df_all[0][1][0].split(', ')
  df_club = df_all[1]
  if len(df_club) <= 15:
    return None, _, _

  df_club.insert(1, "Club", club_name, True)
  df_club.insert(2, "League", league, True)
  df_club.drop(["Номер"], axis=1, inplace=True)

  return df_club, city, country


In [ ]:
hrefs_clubs = gethrefs(url_name="https://www.sports.ru/football/club/?page=", num=1)
import re
club_names = []
for s in hrefs_clubs:
  keyword = ".ru/"
  before_keyword, keyword, after_keyword = s.partition(keyword)
  club_names.append(after_keyword)

# tournament_name
links_clubs = dict()
for club_name in club_names:
  href_tournament = gethrefs_tournament(url_name="https://www.sports.ru/{}stat".format(club_name))
  links_clubs[club_name] = href_tournament


100%|██████████| 2/2 [00:03<00:00,  1.92s/it]


In [ ]:
table_clubs = []

for links in tqdm.tqdm(links_clubs.values()):
  for link_season in links:
    print(link_season)
    if link_season[-7:-1] != '/stat/':
      club_df, city, country = parse_club_page(link_season)
    if club_df is not None:
      table_clubs.append(club_df)

res_clubs = pd.concat(table_clubs, axis=0, ignore_index=True)
res_clubs

  0%|          | 0/50 [00:00<?, ?it/s]

https://www.sports.ru/barcelona/stat/la-liga/
https://www.sports.ru/barcelona/stat/ucl/
https://www.sports.ru/barcelona/stat/supercup-spain/
https://www.sports.ru/barcelona/stat/copa-del-rey/
https://www.sports.ru/barcelona/stat/2020-2021/la-liga/
https://www.sports.ru/barcelona/stat/2019-2020/la-liga/
https://www.sports.ru/barcelona/stat/2018-2019/la-liga/
https://www.sports.ru/barcelona/stat/2017-2018/la-liga/
https://www.sports.ru/barcelona/stat/2016-2017/la-liga/
https://www.sports.ru/barcelona/stat/2015-2016/la-liga/
https://www.sports.ru/barcelona/stat/2014-2015/la-liga/
https://www.sports.ru/barcelona/stat/2013-2014/la-liga/
https://www.sports.ru/barcelona/stat/2012-2013/la-liga/
https://www.sports.ru/barcelona/stat/2011-2012/la-liga/
https://www.sports.ru/barcelona/stat/2010-2011/la-liga/
https://www.sports.ru/barcelona/stat/2009-2010/la-liga/
https://www.sports.ru/barcelona/stat/2008-2009/la-liga/
https://www.sports.ru/barcelona/stat/2007-2008/la-liga/
https://www.sports.ru/ba

  2%|▏         | 1/50 [01:16<1:02:44, 76.83s/it]

https://www.sports.ru/mu/stat/epl/
https://www.sports.ru/mu/stat/ucl/
https://www.sports.ru/mu/stat/league-cup/
https://www.sports.ru/mu/stat/english-cup/
https://www.sports.ru/mu/stat/151758525/
https://www.sports.ru/mu/stat/2020-2021/epl/
https://www.sports.ru/mu/stat/2019-2020/epl/
https://www.sports.ru/mu/stat/2018-2019/epl/
https://www.sports.ru/mu/stat/2017-2018/epl/
https://www.sports.ru/mu/stat/2016-2017/epl/
https://www.sports.ru/mu/stat/2015-2016/epl/
https://www.sports.ru/mu/stat/2014-2015/epl/
https://www.sports.ru/mu/stat/2013-2014/epl/
https://www.sports.ru/mu/stat/2012-2013/epl/
https://www.sports.ru/mu/stat/2011-2012/epl/
https://www.sports.ru/mu/stat/2010-2011/epl/
https://www.sports.ru/mu/stat/2009-2010/epl/
https://www.sports.ru/mu/stat/2008-2009/epl/
https://www.sports.ru/mu/stat/2007-2008/epl/
https://www.sports.ru/mu/stat/2006-2007/epl/
https://www.sports.ru/mu/stat/2005-2006/epl/
https://www.sports.ru/mu/stat/2004-2005/epl/
https://www.sports.ru/mu/stat/2003-2004

  4%|▍         | 2/50 [03:11<1:10:33, 88.21s/it]

...

https://www.sports.ru/lyon/stat/ligue-1/
https://www.sports.ru/lyon/stat/coupe-de-la-ligue/
https://www.sports.ru/lyon/stat/coupe-de-france/
https://www.sports.ru/lyon/stat/2020-2021/ligue-1/
https://www.sports.ru/lyon/stat/2019-2020/ligue-1/
https://www.sports.ru/lyon/stat/2018-2019/ligue-1/
https://www.sports.ru/lyon/stat/2017-2018/ligue-1/
https://www.sports.ru/lyon/stat/2016-2017/ligue-1/
https://www.sports.ru/lyon/stat/2015-2016/ligue-1/
https://www.sports.ru/lyon/stat/2014-2015/ligue-1/
https://www.sports.ru/lyon/stat/2013-2014/ligue-1/
https://www.sports.ru/lyon/stat/2012-2013/ligue-1/
https://www.sports.ru/lyon/stat/2011-2012/ligue-1/
https://www.sports.ru/lyon/stat/2010-2011/ligue-1/
https://www.sports.ru/lyon/stat/2009-2010/ligue-1/
https://www.sports.ru/lyon/stat/2008-2009/ligue-1/
https://www.sports.ru/lyon/stat/2007-2008/ligue-1/
https://www.sports.ru/lyon/stat/2006-2007/ligue-1/
https://www.sports.ru/lyon/stat/2005-2006/ligue-1/
https://www.sports.ru/lyon/stat/2004-2005/l

 98%|█████████▊| 49/50 [53:47<01:02, 62.12s/it]

https://www.sports.ru/orenburg/stat/fnl/
https://www.sports.ru/orenburg/stat/pfl-cup/
https://www.sports.ru/orenburg/stat/cup-russia/
https://www.sports.ru/orenburg/stat/club-friendlies/
https://www.sports.ru/orenburg/stat/2020-2021/fnl/
https://www.sports.ru/orenburg/stat/2017-2018/fnl/
https://www.sports.ru/orenburg/stat/2015-2016/fnl/
https://www.sports.ru/orenburg/stat/2014-2015/fnl/
https://www.sports.ru/orenburg/stat/2013-2014/fnl/
https://www.sports.ru/orenburg/stat/2011-2012/fnl/


100%|██████████| 50/50 [54:12<00:00, 65.05s/it]


,Club,League,Игрок,М,Мин,З,ВнЗ,Г,Пен,П,Г+П,ЖК,КК
0,Барселона,Испания. Ла Лига 2020/2021,Педри,35.0,2317,20,9,2,0,3,5,2,0
1,Барселона,Испания. Ла Лига 2020/2021,Френки де Йонг,35.0,2996,5,2,3,0,4,7,4,0
2,Барселона,Испания. Ла Лига 2020/2021,Антуан Гризманн,33.0,2382,18,4,12,1,7,19,4,0
3,Барселона,Испания. Ла Лига 2020/2021,Лионель Месси,33.0,2844,1,2,28,3,9,37,4,0
4,Барселона,Испания. Ла Лига 2020/2021,Серджи Бускетс,33.0,2266,19,4,0,0,4,4,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20305,Оренбург,Россия. Олимп-ФНЛ 2011/2012,Николай Иванников,6.0,129,0,6,0,0,0,0,0,0
20306,Оренбург,Россия. Олимп-ФНЛ 2011/2012,Олег Кожанов,6.0,359,3,2,0,0,0,0,0,0
20307,Оренбург,Россия. Олимп-ФНЛ 2011/2012,Владимир Щербо,5.0,206,1,3,0,0,0,0,0,0
20308,Оренбург,Россия. Олимп-ФНЛ 2011/2012,Виктор Карпухин,2.0,40,0,2,0,0,0,0,0,0


In [ ]:
res_clubs.to_csv('res_clubs.csv', index=False, encoding='utf-8-sig')